# Prev

## include

In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

## model

In [2]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

  0.000008 seconds (5 allocations: 208 bytes)


ModelSet(TimeModel(150000.0, 0.5, 100, 50), InputModel(0.8, "../../data/init/Sphere"), OutputModel("Test_1", ""))

# Run

## Some Founded Parameters

### Cubic

In [3]:
# # Complete (Oriola Contractile Model) -> Normal Vectors of distance between cells without area
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.162);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.166);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.164);

## Fusion Agg

In [4]:
# Run Model
Par1, Par2 = Cubic(0.00,2.0,3.0), ContractilePar(0.005);
# ContractilePar(0.02);
# RunFusionAggregates(model::ModelSet, Par1, Par2, 15)
size_agg = 15

15

In [5]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(size_agg).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

run_test(agg, model,"Run One Aggregate     ")

Run One Aggregate     ...   0%|                          |  ETA: 24.35 days

ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.


LoadError: CUDA error: an illegal instruction was encountered (code 715, ERROR_ILLEGAL_INSTRUCTION)

In [ ]:
# position=agg.Position
# open("init_stable.xyz", "a") do f
#     write(f, "Initial Stable\n")
#     write(f, "t=0\n")
#     writedlm(f,hcat(agg.Geometry.outline,Matrix(position)), ' ')
# end

# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType("HEK_1", InteractionPar(Par1, Par2),position)], 
#     model
# )
# run_test(agg, model, "Fusion of Two Aggregates")

display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.00754677  -0.0109328     0.0155017
  0.0145561    0.000775563   0.0145104
  0.00466775  -0.0109228    -0.0162079
 -0.0122329    0.0150488     0.00137917
 -0.00939092   0.0139521     0.0065772
  0.00464883  -0.0196339     0.000889229
 -0.0108412    0.00297765   -0.0158262
  0.00817304   0.00804603    0.0169184
  0.0089053   -0.0163785    -0.00749675
 -0.00813834  -0.00761652    0.0179165
 -0.0101483    0.00222923    0.0168669
  0.00451972   0.0149799    -0.0155137
  0.00757989   0.00994894    0.0148586
  ⋮                         
  0.0165459   -0.00625554    0.00109714
 -0.00919354   0.0149862    -0.00947685
 -0.00573339  -0.0160227     0.00647726
  0.0107153   -0.016373     -0.00366959
  0.0192991    0.00570937   -0.00123587
  0.0101462    0.0127428    -0.0127152
  0.0138104    0.015113     -0.00749557
  0.0190368   -0.000481036   0.000697486
 -0.0138346    0.0128122     0.00196324
  0.015493     0.0120841     0.00872584
  0.01276

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.99829    -5.08042    -13.3494
  0.0309798  -5.39805    -13.2906
  2.18577    -5.3081     -13.3017
 -5.00129    -2.86012    -13.2496
 -3.00259    -3.22445    -13.1577
 -0.968686   -3.44879    -13.2099
  0.998308   -3.70337    -13.2634
  3.17346    -3.56661    -13.3628
 -5.79959    -1.00431    -13.2308
 -3.85856    -1.24683    -13.3202
 -1.94857    -1.54889    -13.3485
  0.0414279  -1.78762    -13.2583
  2.01535    -1.98281    -13.2795
  ⋮                      
 -1.36326     1.01698     13.2142
  0.534336    0.763028    13.1868
  2.35922     0.380751    13.1239
  4.26119     0.0712582   13.0311
 -4.34156     3.27968     13.209
 -2.37386     2.94051     13.2557
 -0.45656     2.68488     13.1146
  1.56126     2.3885      13.1421
  3.45335     2.10515     12.9919
 -1.37021     4.59056     13.0811
  0.63275     4.34949     13.0875
  2.64439     3.95858     13.126